In [1]:
import argparse
import os

from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from datetime import datetime

from ego_rate_estimator.cpr_rate_detection_video import ego_rate_detect, init_models
from smartwatch_rate_estimator.cpr_rate_detection_smartwatch import smartwatch_rate_detect, get_gt_cpr_rate

import cv2

from PIL import Image

import matplotlib.pyplot as plt

import argparse
import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



/sfs/gpfs/tardis/home/cjh9fw/Desktop/2024/repos/EgoExoEMS/Benchmarks/CPR_quality/late_fusion


Importing from timm.models.layers is deprecated, please import via timm.layers


trying to load grounding dino directly


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)


final text_encoder_type: bert-base-uncased
Models initialized.


I0000 00:00:1744982254.148104  670823 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1744982254.225665  671054 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.54.14), renderer: NVIDIA A40/PCIe/SSE2


In [2]:
# get cmd line args
args = DefaultArgsNamespace()

keysteps = args.dataloader_params['keysteps']
out_classes = len(keysteps)

modality = args.dataloader_params['modality']
print("Modality: ", modality)
print("Num of classes: ", out_classes)

window = args.dataloader_params['observation_window']
print("Window: ", window)

task = args.dataloader_params['task']
print("Task: ", task)


# train_loader, val_loader, test_loader = get_dataloaders(args)
train_loader, val_loader, test_loader, train_class_stats, val_class_stats = eee_get_dataloaders(args)
args.dataloader_params['train_class_stats'] = train_class_stats
args.dataloader_params['val_class_stats'] = val_class_stats

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Modality:  ['video', 'smartwatch', 'depth_sensor']
Num of classes:  35
Window:  150
Task:  cpr_quality
********** ========== **********
Loading dataloader for CPR quality task
Train class stats:  {'chest_compressions': 215}
Train Number of classes:  1
val class stats:  {'chest_compressions': 49}
Val Number of classes:  1
train dataset size:  215
val dataset size:  49
test dataset size:  121


W0000 00:00:1744982254.259969  671004 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744982254.269718  671025 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Functions for rate detection

In [ ]:
def detect_rate(frames,smartwatch,gt,window_size,video_id):
    # iterate through the frames using a tumbling window with the given window size
    # and calculate the detection rate
    # for each window

    for i in range(0, len(frames), window_size):
        print("---------------------")
        print("Window: ", i)

        # get the current window
        window = frames[i:i+window_size]
        # get the current smartwatch data
        smartwatch_window = smartwatch[i:i+window_size]
        # get the current gt data
        gt_window = gt[i:i+window_size]

        print("Window: ", window.shape)
        print("Smartwatch: ", smartwatch_window.shape)
        print("GT: ", gt_window.shape)

        # call video rate detection function
        ego_cpr_rate = ego_rate_detect(window,video_id)
        
        # call smartwatch rate detection function
        smartwatch_cpr_rate = smartwatch_rate_detect(smartwatch_window)
        print("Smartwatch CPR Rate: ", smartwatch_cpr_rate)

        gt_cpr_rate = get_gt_cpr_rate(gt_window)
        print("GT CPR Rate: ", gt_cpr_rate)

        # weighted fusion of smartwatch and video rate to achieve best performance
        # final_predicted_cpr_rate = 0.5 * smartwatch_cpr_rate + 0.5 * ego_cpr_rate

    return None,None,None,None


    
    

In [4]:
for i, batch in enumerate(train_loader):
    print("****************")
    print("Batch: ", i)

    input,feature_size, gt_sensor_data = preprocess(batch, modality, None, device, task=task)
    print("Subject: ", batch['subject_id'])
    print("Trial", batch['trial_id'])
    print("Input frames: ", input['frames'].shape)

    video_id = f"{batch['subject_id']}_{batch['trial_id']}_ego"
    fused_rate, sw_rate, video_rate, gt_rates = detect_rate(input['frames'][0], input['smartwatch'][0], gt_sensor_data[0], window, video_id)
    print("****************")

    break
    
    # # sample image
    # image = input['frames'][0][0].permute(1,2,0).cpu().numpy()
    # # convert to PIL image
    # image = Image.fromarray(image)
    # # show pil image
    # plt.imshow(image)
    
    # print("Input smartwatch: ", input['smartwatch'].shape)
    # print("gt_sensor_data shape: ", gt_sensor_data.shape)
    
    # # plot the smartwatch data (3-axis accelerometer)
    # smartwatch_data = input['smartwatch'][0].cpu().numpy()
    # plt.figure(figsize=(10, 5))
    # plt.plot(smartwatch_data[:, 0], label='X-axis')
    # plt.plot(smartwatch_data[:, 1], label='Y-axis')
    # plt.plot(smartwatch_data[:, 2], label='Z-axis')
    # plt.title('Smartwatch Data')
    # plt.xlabel('Time')
    # plt.ylabel('Acceleration')
    # plt.legend()
    # plt.show()
    
    # # plot the gt_sensor_data (1-axis distance)
    # gt_sensor_data = gt_sensor_data[0].cpu().numpy()
    # plt.figure(figsize=(10, 5))
    # plt.plot(gt_sensor_data, label='Distance')
    # plt.title('Ground Truth Sensor Data')
    # plt.xlabel('Time')
    # plt.ylabel('Distance')
    # plt.legend()
    # plt.show()
    # print("****************")
    
    # break
    

****************
Batch:  0
Special case for CPR quality task
Subject:  ['ng4']
Trial ['1']
Input frames:  torch.Size([1, 207, 3, 224, 224])
---------------------
Window:  0
Window:  torch.Size([150, 3, 224, 224])
Smartwatch:  torch.Size([150, 3])
GT:  torch.Size([150, 1])
Starting ego rate detection...
Number of RGB images:  (150, 224, 224, 3)


The `device` argument is deprecated and will be removed in v5 of Transformers.
torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
None of the inputs have requires_grad=True. Gradients will be None
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
W0000 00:00:1744907991.850649  184908 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Number of matching RGB frames:  150
Number of wrist keypoints:  150
number of filtered wrist keypoints:  113
number of filtered rgb images in the window:  113
Predicted CPR rate per window: 6.5 cycles
Predicted CPR rate (BPM): 78.0
Smartwatch CPR Rate:  192.0
GT CPR Rate:  144.0
---------------------
Window:  150
Window:  torch.Size([57, 3, 224, 224])
Smartwatch:  torch.Size([57, 3])
GT:  torch.Size([57, 1])
Starting ego rate detection...
Number of RGB images:  (57, 224, 224, 3)
Number of matching RGB frames:  57
Number of wrist keypoints:  57
number of filtered wrist keypoints:  24
number of filtered rgb images in the window:  24
Predicted CPR rate per window: 3.0 cycles
Predicted CPR rate (BPM): 36.0
Smartwatch CPR Rate:  142.10526315789474
GT CPR Rate:  126.31578947368422
****************
